# R Snowflake Connectivity

This notebook covers **connecting to Snowflake from R** in Workspace Notebooks.

**Prerequisites:** Run `r_setup_interop.ipynb` Section 1 first to install R.

**Connection Methods:**

| Method | Auth | Query Pushdown | Best For |
|--------|------|----------------|----------|
| **Reticulate + Snowpark** | Session token | ✅ Yes | Easiest - no setup |
| **ADBC + adbcsnowflake** | PAT token | ✅ Yes | Direct R queries |
| **DuckDB + Snowflake ext** | Secret | ✅ Yes | Local caching + dplyr |
| **Iceberg REST** | PAT/JWT | ❌ No | Direct file access |

**Quick Start:**
1. Run Section 1 (Prerequisites)
2. Try Section 3 (Reticulate) first - it works immediately
3. Explore other methods as needed

---

# Section 1: Prerequisites

**Before using this notebook:**

1. Run `r_setup_interop.ipynb` Section 1 to install R and rpy2
2. Ensure you have database access (most methods use SNOWFLAKE_SAMPLE_DATA)

The cells below set up helper functions and verify R is available.

In [ ]:
# Setup R helpers
import sys
sys.path.insert(0, '.')  # Ensure current directory is in path

from r_helpers import setup_r_environment

result = setup_r_environment()

if result['success']:
    print("✓ R environment configured successfully")
    print(f"  R version: {result['r_version']}")
    print(f"  rpy2 installed: {result['rpy2_installed']}")
    print(f"  %%R magic registered: {result['magic_registered']}")
else:
    print("✗ Setup failed:")
    for error in result['errors']:
        print(f"  - {error}")

In [ ]:
%%R
# Print R version (simple output works fine)
R.version.string

In [ ]:
from r_helpers import check_environment, print_diagnostics

# Run and display diagnostics
print_diagnostics()

---

# Section 2: Snowflake Database Connectivity

This section demonstrates connecting to Snowflake from R using ADBC.

**Prerequisites:**
- Run the setup script with `--adbc` flag (Section 1.1)
- Have appropriate Snowflake permissions

## Authentication Options

| Method | Status | Notes |
|--------|--------|-------|
| Python `get_active_session()` | ✅ Works | Use for Snowpark queries, bridge to R via rpy2 |
| ADBC with PAT | ✅ Works | Direct R-to-Snowflake, requires PAT token |
| SPCS OAuth Token | ❌ Blocked | Container token not authorized for ADBC |
| Username/Password | ❌ Blocked | SPCS requires OAuth |

## Connection Management

This notebook uses connection pooling - the ADBC connection is stored as `r_sf_con` in R's global environment and reused across cells. This avoids the overhead of creating new connections for each query.

## 2.1 Setup Python Session

This cell loads configuration and establishes the Snowflake session.

**Configuration:**
- Copy `notebook_config.yaml.template` to `notebook_config.yaml`
- Edit with your account details (for Local IDE)
- The config provides database, schema, warehouse, and query settings

**Environments:**
- **Workspace Notebook**: Uses `get_active_session()` (built-in OAuth)
- **Local IDE (VSCode/Cursor)**: Uses config file for key-pair auth

In [ ]:
# Setup Snowflake session and load configuration
import os
import sys

# =============================================================================
# Load Configuration File
# =============================================================================
CONFIG_FILE = 'notebook_config.yaml'
CONFIG_TEMPLATE = 'notebook_config.yaml.template'

def load_config():
    """Load configuration from YAML file."""
    try:
        import yaml
    except ImportError:
        print("Installing PyYAML...")
        import subprocess
        subprocess.run([sys.executable, "-m", "pip", "install", "pyyaml", "-q"], check=True)
        import yaml
    
    if os.path.exists(CONFIG_FILE):
        with open(CONFIG_FILE) as f:
            config = yaml.safe_load(f)
        print(f"✓ Loaded config from {CONFIG_FILE}")
        return config
    elif os.path.exists(CONFIG_TEMPLATE):
        print(f"✗ Config not found!")
        print(f"  Copy {CONFIG_TEMPLATE} to {CONFIG_FILE} and customize.")
        return {}
    else:
        print("✗ No config file found, using defaults")
        return {}

CONFIG = load_config()

# Extract config sections for easy access
CONN_CONFIG = CONFIG.get('connection', {})
DEFAULTS = CONFIG.get('defaults', {})
QUERY_CONFIG = CONFIG.get('sample_queries', {})
ICEBERG_CONFIG = CONFIG.get('iceberg', {})

# =============================================================================
# Environment Detection and Session Setup
# =============================================================================
def detect_environment():
    """
    Detect if running in Snowflake Workspace Notebook or local IDE.
    Returns: ('workspace', session) or ('local', config_dict)
    """
    workspace_indicators = [
        os.path.exists('/snowflake/session/token'),
        'SNOWFLAKE_HOST' in os.environ,
        '/home/udf' in os.getcwd(),
    ]
    
    if any(workspace_indicators):
        try:
            from snowflake.snowpark.context import get_active_session
            session = get_active_session()
            return ('workspace', session)
        except Exception as e:
            return ('workspace_error', str(e))
    else:
        return ('local', None)

# Detect environment
ENV_TYPE, ENV_RESULT = detect_environment()

if ENV_TYPE == 'workspace':
    session = ENV_RESULT
    
    # Get connection details from session, with config overrides
    ACCOUNT = session.sql('SELECT CURRENT_ACCOUNT()').collect()[0][0]
    USER = session.sql('SELECT CURRENT_USER()').collect()[0][0]
    DATABASE = DEFAULTS.get('database') or session.get_current_database()
    SCHEMA = DEFAULTS.get('schema') or session.get_current_schema()
    WAREHOUSE = DEFAULTS.get('warehouse') or session.get_current_warehouse()
    ROLE = session.get_current_role()
    
    # Build unified config
    ENV_CONFIG = {
        'account': ACCOUNT,
        'user': USER,
        'database': DATABASE,
        'schema': SCHEMA,
        'warehouse': WAREHOUSE,
        'role': ROLE,
    }
    
    print(f"\nEnvironment: Workspace Notebook")
    print(f"  Account:   {ACCOUNT}")
    print(f"  User:      {USER}")
    print(f"  Role:      {ROLE}")
    print(f"  Database:  {DATABASE}")
    print(f"  Schema:    {SCHEMA}")
    print(f"  Warehouse: {WAREHOUSE}")
    
elif ENV_TYPE == 'local':
    session = None
    
    # Use config file values for local IDE
    ENV_CONFIG = {
        'account': CONN_CONFIG.get('account', '<YOUR_ACCOUNT>'),
        'user': CONN_CONFIG.get('user', '<YOUR_USER>'),
        'database': DEFAULTS.get('database', 'SNOWFLAKE_SAMPLE_DATA'),
        'schema': DEFAULTS.get('schema', 'TPCH_SF1'),
        'warehouse': DEFAULTS.get('warehouse', '<YOUR_WAREHOUSE>'),
        'role': DEFAULTS.get('role', 'PUBLIC'),
        'private_key_path': CONN_CONFIG.get('private_key_path', '~/.ssh/snowflake_rsa_key.p8'),
    }
    
    print(f"\nEnvironment: Local IDE")
    print(f"  Account:   {ENV_CONFIG['account']}")
    print(f"  User:      {ENV_CONFIG['user']}")
    print(f"  Database:  {ENV_CONFIG['database']}")
    print(f"  Warehouse: {ENV_CONFIG['warehouse']}")
    print(f"  Key path:  {ENV_CONFIG['private_key_path']}")
    
    if '<YOUR_' in str(ENV_CONFIG.values()):
        print("\n⚠️  Some config values need to be set!")
        print(f"   Edit {CONFIG_FILE} with your values")
else:
    print(f"Warning: Environment detection issue: {ENV_RESULT}")
    ENV_CONFIG = {}
    session = None

# Make query config easily accessible
ROW_LIMIT = QUERY_CONFIG.get('default_row_limit', 1000)
LARGE_ROW_LIMIT = QUERY_CONFIG.get('large_row_limit', 10000)
SAMPLE_START_DATE = QUERY_CONFIG.get('sample_start_date', '1995-01-01')
TABLES = QUERY_CONFIG.get('tables', {'nation': 'NATION', 'customer': 'CUSTOMER', 'orders': 'ORDERS'})

### Authentication Methods Overview

This notebook supports multiple authentication methods for different connectivity approaches:

| Section | Method | Auth Type | Status | Environment |
|---------|--------|-----------|--------|-------------|
| **3.1** | `get_active_session()` | Built-in OAuth | ✅ Working | Workspace |
| **3.3-3.6** | ADBC + PAT | PAT Token | ✅ Working | Workspace |
| **4.2** | ADBC + Key Pair | JWT | ✅ Working | Both |
| **5** | Reticulate | Session OAuth | ✅ Working | Workspace |
| **7** (DuckDB) | Key Pair | JWT | ✅ Working | Local IDE |
| **7.3.1** | Python Bridge | Session OAuth | ✅ Working | Both |
| **8** | Horizon Catalog | JWT | ✅ Working | Local IDE |

**Recommended Path for Most Users:**
1. Run **Section 3.1** (required - sets up session)
2. Choose ONE of:
   - **Section 5** (Reticulate) - Easiest, uses built-in auth, works everywhere
   - **Section 7.3.1** (Python Bridge) - For dplyr workflows, works everywhere  
   - **Section 7** (DuckDB Direct) - For Local IDE only (key-pair required)
   - **Section 3** (ADBC) - For direct R-to-Snowflake in Workspace

## 2.2 Create Programmatic Access Token (PAT) (For ADBC - Optional)

**Used by:** Section 3 (R-ADBC) only. Skip if using Section 5 (Reticulate) or Section 7 (DuckDB).

PAT enables direct R-to-Snowflake ADBC connections. The session from Section 3.1 is used to create the token.

In [ ]:
# Create PAT for authentication (requires session from 3.1)
from r_helpers import PATManager

# Uses the 'session' variable from Section 3.1
if session is None:
    print("Please run Section 3.1 first!")
else:
    pat_manager = PATManager(session)
    pat_result = pat_manager.create_pat()  # Creates PAT with 1 day expiry
    
    if pat_result['success']:
        print(f"✓ PAT created successfully")
        print(f"  Token: {pat_result['token'][:20]}...")
        print(f"  Expires: {pat_result['expires_at']}")
    else:
        print(f"✗ PAT creation failed: {pat_result.get('error', 'Unknown error')}")

In [ ]:
# Check PAT status at any time
status = pat_mgr.get_status()
print("PAT Status:")
for key, value in status.items():
    print(f"  {key}: {value}")

## 2.3 Validate ADBC Prerequisites

Before connecting, validate that all ADBC prerequisites are met.

In [ ]:
from r_helpers import validate_adbc_connection

valid, message = validate_adbc_connection()
print(message)

## 2.4 Initialize R Connection Management

Load the connection management functions into R. This provides:
- `get_snowflake_connection()` - Get or create connection (stored as `r_sf_con`)
- `close_snowflake_connection()` - Close and release connection
- `is_snowflake_connected()` - Check connection status
- `snowflake_connection_status()` - Get detailed status

In [ ]:
from r_helpers import init_r_connection_management

success, msg = init_r_connection_management()
print(msg)

## 2.5 Connect to Snowflake from R (ADBC)

Use `get_snowflake_connection()` to establish or reuse the ADBC connection.

The connection is stored as `r_sf_con` in R's global environment and is automatically reused in subsequent cells.

In [ ]:
%%R
# Get or create the Snowflake connection
# Connection is stored globally as r_sf_con
r_sf_con <- get_snowflake_connection()

# Show connection status (uses print_connection_status() for clean output)
print_connection_status()

## 2.6 Query Snowflake from R

Run queries using the `r_sf_con` connection. The connection is automatically reused across cells.

In [ ]:
%%R
# Simple test query using r_sf_con
r_sf_con |>
  read_adbc("SELECT CURRENT_USER() AS USER, CURRENT_ROLE() AS ROLE, CURRENT_WAREHOUSE() AS WAREHOUSE") |>
  tibble::as_tibble()

In [ ]:
%%R
# Query sample data from Snowflake
# Using the shared SNOWFLAKE_SAMPLE_DATA database
nations <- r_sf_con |>
  read_adbc("
    SELECT N_NATIONKEY, N_NAME, N_REGIONKEY 
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION 
    ORDER BY N_NATIONKEY
    LIMIT 10
  ") |>
  tibble::as_tibble()

nations

In [ ]:
%%R
# More complex query with aggregation
library(dplyr)

orders_summary <- r_sf_con |>
  read_adbc("
    SELECT 
      O_ORDERSTATUS,
      COUNT(*) as ORDER_COUNT,
      SUM(O_TOTALPRICE) as TOTAL_VALUE,
      AVG(O_TOTALPRICE) as AVG_VALUE
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
    GROUP BY O_ORDERSTATUS
    ORDER BY ORDER_COUNT DESC
  ") |>
  tibble::as_tibble()

orders_summary

## 2.7 dplyr with Snowflake via ADBC (Future)

> **⚠️ NOT WORKING YET** - This section documents the intended approach, but it's blocked by missing driver methods.

### The Goal

R users want dplyr syntax with lazy evaluation and query pushdown:

```r
tbl(con, "CUSTOMER") %>%
  filter(C_MKTSEGMENT == "BUILDING") %>%
  collect()  # SQL runs on Snowflake
```

### Current Blockers

1. **adbcsnowflake** driver missing `GetParameterSchema` method ([apache/arrow-adbc](https://github.com/apache/arrow-adbc))
2. **dbplyr** ADBC backend in development ([tidyverse/dbplyr#1787](https://github.com/tidyverse/dbplyr/issues/1787))

### Working Alternative

Use the **hybrid approach** in Section 7.3.1: fetch via ADBC → load to DuckDB → use dplyr locally.

### Track Progress

- [tidyverse/dbplyr#1787](https://github.com/tidyverse/dbplyr/issues/1787) - Hadley's ADBC backend work
- [apache/arrow-adbc](https://github.com/apache/arrow-adbc) - Snowflake driver updates

In [ ]:
%%R
# NOT WORKING YET - adbi + adbcsnowflake blocked by missing GetParameterSchema
# See markdown cell above for details
#
# When the ecosystem is ready, this will work:
#
# library(adbi)
# library(DBI)
# library(dplyr)
# library(dbplyr)
#
# con <- dbConnect(
#     adbi::adbi("adbcsnowflake"),
#     username = Sys.getenv("SNOWFLAKE_USER"),
#     `adbc.snowflake.sql.account` = Sys.getenv("SNOWFLAKE_ACCOUNT"),
#     `adbc.snowflake.sql.auth_type` = "auth_pat",
#     `adbc.snowflake.sql.client_option.auth_token` = Sys.getenv("SNOWFLAKE_PAT"),
#     ...
# )
#
# # Then use dplyr!
# tbl(con, "CUSTOMER") %>%
#     filter(C_MKTSEGMENT == "BUILDING") %>%
#     collect()

cat("adbi + dbplyr for Snowflake is not yet working.\n")
cat("Use Section 7.3.1 (Python Bridge) or hybrid approach instead.\n")

In [ ]:
%%R
# Verify connection is being reused (not recreated)
cat("Connection still valid:", is_snowflake_connected(), "\n")

## 2.8 Query from Python, Analyze in R

An alternative pattern: use Python's Snowpark session for querying, then pass data to R for analysis.

In [ ]:
# Query Snowflake via Python
customers_df = session.sql(f"""
    SELECT 
        C_CUSTKEY,
        C_NAME,
        C_NATIONKEY,
        C_ACCTBAL
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
    LIMIT 100
""").to_pandas()

print(f"Retrieved {len(customers_df)} rows")
customers_df.head()

In [ ]:
%%R -i customers_df
# Analyze the data in R
library(dplyr)

cat("Summary Statistics for Customer Account Balance:\n")
rprint(summary(customers_df$C_ACCTBAL))

cat("\nCustomers by Nation (top 5):\n")
result <- customers_df %>%
  group_by(C_NATIONKEY) %>%
  summarise(
    count = n(),
    avg_balance = mean(C_ACCTBAL),
    total_balance = sum(C_ACCTBAL)
  ) %>%
  arrange(desc(count)) %>%
  head(5)

rprint(result)  # Use rprint() for clean output

## 2.9 Check Connection Status

You can check the connection status from either Python or R.

In [ ]:
# Check status from Python
from r_helpers import get_r_connection_status

status = get_r_connection_status()
print("R Connection Status (from Python):")
for key, value in status.items():
    print(f"  {key}: {value}")

In [ ]:
%%R
# Get or create the Snowflake connection
# Connection is stored globally as r_sf_con
r_sf_con <- get_snowflake_connection()

# Show connection status (uses print_connection_status() for clean output)
print_connection_status()

## 2.9 Clean Up

Close ADBC connection and optionally remove the PAT.

In [ ]:
%%R
# Close the Snowflake connection
close_snowflake_connection()

In [ ]:
# Alternative: Close from Python
# from r_helpers import close_r_connection
# success, msg = close_r_connection()
# print(msg)

In [ ]:
# Cleanup - remove PAT
# pat_mgr.remove_pat()
# print("PAT removed")

---

# Section 3: Alternative Authentication - Key Pair (JWT)

This section demonstrates Key Pair (JWT) authentication as an alternative to PAT.

## Authentication Methods for R ADBC

| Method | Status | Notes |
|--------|--------|-------|
| **PAT (Programmatic Access Token)** | ✅ Working | **Recommended** - easiest to set up (see Section 3) |
| **Key Pair (JWT)** | ✅ Working | Alternative - no token expiry, shown below |
| SPCS OAuth Token | ❌ Blocked | Container token restricted to specific connectors |
| Username/Password | ❌ Blocked | SPCS enforces OAuth for internal connections |

> **Note:** For tests of non-working methods, see `archive/auth_methods_not_working.ipynb`

## Prerequisites

- ADBC installed (`--adbc` flag during setup)
- RSA key pair generated
- Public key registered with your Snowflake user

## 3.1 Load Alternative Auth Test Functions

Load the R functions for testing different authentication methods.

In [ ]:
from r_helpers import init_r_alt_auth

success, msg = init_r_alt_auth()
print(msg)

## 3.2 Key Pair (JWT) Authentication (Alternative for ADBC)

**Used by:** Section 3 (R-ADBC) as an alternative to PAT, and Section 8 (Iceberg) for Horizon Catalog API.

Key pair authentication uses RSA keys instead of passwords/PAT. This method is MFA-compatible and doesn't expire like PAT tokens.

In [ ]:
# Key-pair authentication setup
from r_helpers import KeyPairAuth

# Initialize key pair auth helper
kp_auth = KeyPairAuth()

# Generate a new key pair (or use load_private_key() for existing key)
# Note: Requires 'cryptography' package: pip install cryptography
result = kp_auth.generate_key_pair(
    key_size=2048,
    output_dir="/tmp",
    passphrase=None  # Set a passphrase for encrypted key
)

if result['success']:
    print("✓ Key pair generated successfully")
    print(f"  Private key: {result['private_key_path']}")
    print(f"  Public key:  {result['public_key_path']}")
    print(f"\n  Public key for Snowflake registration:")
    print(f"  {result['public_key_for_snowflake'][:50]}...")
else:
    print(f"✗ Key generation failed: {result['error']}")

### Step 2: Register Public Key with Snowflake

Run this SQL to register the public key with your user (requires ACCOUNTADMIN or appropriate privileges).

In [ ]:
# Generate key registration SQL
if result['success']:
    sql = kp_auth.register_public_key_sql(result['public_key_for_snowflake'])
    print("Run this SQL to register the public key:")
    print("-" * 60)
    print(sql)
    print("-" * 60)
    print("\nOr run via Snowpark session:")
    print("  session.sql(sql).collect()")

In [ ]:
# Register public key in Snowflake
session.sql(sql).collect()

### Step 3: Configure and Test Key Pair Auth

In [ ]:
# Configure environment for key pair auth
config = kp_auth.configure_for_adbc()
print("Key Pair Auth Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

In [ ]:
%%R
# Test key pair authentication
# Note: Public key must be registered with user first!
result <- test_keypair_auth()
rprint(result)

## 3.3 Authentication Summary

### Working Methods

| Method | Auth Type | Best For |
|--------|-----------|----------|
| **PAT** | `auth_pat` | Most use cases - easy programmatic setup |
| **Key Pair** | `auth_jwt` | Long-lived credentials without expiry |

### Non-Working Methods (Blocked by SPCS)

| Method | Reason |
|--------|--------|
| SPCS OAuth Token | Restricted to specific Snowflake connectors |
| Username/Password | SPCS enforces OAuth internally |

> See `archive/auth_methods_not_working.ipynb` for test code if needed.

---

# Section 4: Reticulate - Access Snowpark from R

This section demonstrates using **reticulate** to access the Python Snowpark session directly from R. This is an alternative to ADBC that leverages the notebook's built-in authentication.

## Advantages of Reticulate Approach

| Feature | Reticulate + Snowpark | ADBC |
|---------|----------------------|------|
| Authentication | Uses notebook's built-in auth | Requires PAT or Key Pair |
| Setup | No additional auth setup | PAT creation or key registration |
| Connection | Shares Python session | Separate R connection |
| Best for | Quick queries, prototyping | Production R pipelines |

## How It Works

1. R accesses Python's Snowpark session via reticulate
2. Execute SQL queries using `session$sql()`
3. Convert results to pandas DataFrame with `.to_pandas()`
4. Reticulate automatically converts pandas → R data.frame

## Output Pattern

For best display in Notebooks, use `%%R -o variable` to export R data frames to Python, then display them in a subsequent Python cell. This lets the Notebook render the DataFrame with proper formatting.

## 4.1 Setup Reticulate

Configure reticulate to use the notebook's Python environment.

> **Note:** You may see a warning about reticulate/rpy2 compatibility. This is safe to ignore if using reticulate >= 1.25 (installed by default). The issue was fixed in reticulate PR #1188.

In [ ]:
%%R
library(reticulate)

# Use the same Python that's running the notebook kernel
# This ensures we access the same Snowpark session
use_python(Sys.which("python3"), required = TRUE)

# Verify Python is accessible
py_config()

## 4.2 Access Snowpark Session from R

Import the Snowpark module and get the active session. This uses the notebook's built-in authentication - no PAT required!

In [ ]:
%%R
# Import Snowpark module
snowpark <- import("snowflake.snowpark")

# Get the active session (uses notebook's built-in auth)
session <- snowpark$Session$builder$getOrCreate()

# Verify connection
rcat("Connected to Snowflake via Snowpark!")
rcat("Account: ", session$get_current_account())
rcat("User: ", session$get_current_user())
rcat("Database: ", session$get_current_database())
rcat("Schema: ", session$get_current_schema())

## 4.3 Query Snowflake and Get R DataFrame

Execute SQL queries and convert results to R data frames.

**Output Pattern:** Use `%%R -o variable` to export results to Python, then display in the next cell for nice Notebook formatting.

In [ ]:
%%R -o nations_df
# Execute a query and get Snowpark DataFrame
# Use -o to export result to Python for nice display
nations_df <- session$sql("
    SELECT N_NATIONKEY, N_NAME, N_REGIONKEY 
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION 
    LIMIT 10
")$to_pandas()

# Print data type (R sees this as a data.frame)
cat("R data type:", class(nations_df), "\n")

In [ ]:
# Display the exported DataFrame (nice Notebook rendering)
nations_df

## 4.4 R Analysis on Snowflake Data

Perform R analysis using dplyr on data retrieved via Snowpark. Use `-o` to export the result for display.

In [ ]:
%%R -o customer_analysis
# Query customer data with aggregation
customers_df <- session$sql("
    SELECT 
        C_MKTSEGMENT,
        COUNT(*) as CUSTOMER_COUNT,
        AVG(C_ACCTBAL) as AVG_BALANCE,
        MIN(C_ACCTBAL) as MIN_BALANCE,
        MAX(C_ACCTBAL) as MAX_BALANCE
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
    GROUP BY C_MKTSEGMENT
    ORDER BY AVG_BALANCE DESC
")$to_pandas()

# Use dplyr for additional analysis
library(dplyr)

customer_analysis <- customers_df %>%
    mutate(
        BALANCE_RANGE = MAX_BALANCE - MIN_BALANCE,
        SEGMENT_SIZE = case_when(
            CUSTOMER_COUNT > 30000 ~ "Large",
            CUSTOMER_COUNT > 29000 ~ "Medium",
            TRUE ~ "Small"
        )
    )

cat("Analysis complete - result exported to Python\n")

In [ ]:
# Display the R analysis result (exported via -o)
customer_analysis

## 4.5 Helper Function for Snowpark Queries

Create a convenience function to simplify querying. Use `-o` to export results for display.

In [ ]:
%%R -o orders_summary
#' Query Snowflake via Snowpark and return R data.frame
#' 
#' @param sql SQL query string
#' @return R data.frame with query results
snowpark_query <- function(sql) {
    session$sql(sql)$to_pandas()
}

# Example usage - export result with -o
orders_summary <- snowpark_query("
    SELECT 
        O_ORDERSTATUS,
        COUNT(*) as ORDER_COUNT,
        SUM(O_TOTALPRICE) as TOTAL_VALUE
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
    GROUP BY O_ORDERSTATUS
")

cat("Query complete - orders_summary exported to Python\n")

In [ ]:
# Display the orders summary (exported via -o)
orders_summary

## 4.6 Reticulate vs ADBC Comparison

| Aspect | Reticulate + Snowpark | ADBC (Section 3 & 4) |
|--------|----------------------|----------------------|
| **Authentication** | Automatic (notebook's session) | PAT or Key Pair required |
| **Setup complexity** | Minimal | Moderate |
| **Data path** | Snowflake → Snowpark → pandas → R | Snowflake → Arrow → R |
| **Performance** | Good for moderate data | Better for large data (Arrow) |
| **R-native** | No (via Python) | Yes (native R driver) |
| **Best for** | Quick analysis, prototyping | Production R workflows |

### When to Use Each

**Use Reticulate + Snowpark when:**
- You need quick access without auth setup
- Working interactively/prototyping
- Data sizes are moderate (< 1M rows)
- You're already using Python and R together

**Use ADBC when:**
- Building production R pipelines
- Working with large datasets
- Need pure R solution
- Require connection pooling/management

---

# Section 5: DuckDB Integration (Local Caching)

## Goal: dplyr on Snowflake Data

**R users want to write dplyr, not SQL.** This section provides a working approach using DuckDB as a local cache.

### Why DuckDB?

Native `adbi + dbplyr → Snowflake` is **not yet working** (see Section 3.7). The workaround:

```
Snowflake → (ADBC or Python) → DuckDB (local) → dplyr
```

This "hybrid" approach lets you:
- Fetch data once from Snowflake
- Use full dplyr workflows locally on DuckDB
- Get fast iteration without repeated Snowflake queries

### When to Use This Section

- **Local caching** - Fetch once, analyze many times without hitting Snowflake
- **dplyr workflows** - Use familiar dplyr syntax with lazy evaluation on DuckDB
- **Cross-engine queries** - Join Snowflake data with local files/parquet
- **Complex analytics** - Use DuckDB's window functions, spatial extensions, etc.

## Architecture

```
R (dplyr/dbplyr)
    ↕ DBI
DuckDB (in-memory or file)
    ↑ Data loaded via:
      • Python bridge (Section 7.3.1)
      • ADBC fetch → DuckDB insert
Snowflake
```

## 5.1 Prerequisites

**Session Setup**: Environment detection and session setup is now handled in **Section 3.1**.
Make sure you've run that cell before proceeding with DuckDB integration.

The `ENV_TYPE` variable tells you which environment you're in:
- `'workspace'` - Workspace Notebook (uses OAuth)
- `'local'` - Local IDE (uses key-pair auth)

In [ ]:
# Verify session from Section 3.1
if 'ENV_TYPE' not in dir():
    print("Please run Section 3.1 first to set up the session!")
else:
    print(f"Environment: {ENV_TYPE}")
    print(f"Session: {'Available' if session else 'Not available (local mode)'}")
    if ENV_CONFIG:
        print(f"Account: {ENV_CONFIG.get('account', 'N/A')}")

## 5.2 Configure Connection (Local IDE Only)

**Skip this section if running in Workspace Notebook.**

For local IDEs, set these environment variables before starting your notebook:

```bash
export SNOWFLAKE_ACCOUNT="your_account"     # e.g., "xy12345"  
export SNOWFLAKE_USER="your_user"
export SNOWFLAKE_DATABASE="SNOWFLAKE_SAMPLE_DATA"
export SNOWFLAKE_WAREHOUSE="COMPUTE_WH"
export SNOWFLAKE_PRIVATE_KEY_PATH="~/.ssh/snowflake_rsa_key.p8"
```

Then restart your notebook and run Section 3.1 to detect the environment.

In [ ]:
# Local IDE verification (optional)
# Environment variables should be set before starting the notebook
# This cell just displays the current configuration

if ENV_TYPE == 'local':
    print("Local IDE Configuration:")
    for key in ['account', 'user', 'database', 'warehouse', 'private_key_path']:
        print(f"  {key}: {ENV_CONFIG.get(key, 'N/A')}")
    
    # Check if key file exists
    key_path = os.path.expanduser(ENV_CONFIG.get('private_key_path', ''))
    if os.path.exists(key_path):
        print(f"\n✓ Private key file found")
    else:
        print(f"\n✗ Private key file NOT found: {key_path}")
else:
    print("Running in Workspace - no local configuration needed")

## 5.3 DuckDB + Snowflake Setup in R

This section configures DuckDB with the Snowflake extension for direct database connectivity.

### Authentication Options (per [iqea-ai/duckdb-snowflake](https://github.com/iqea-ai/duckdb-snowflake))

| Method | Status | Notes |
|--------|--------|-------|
| **Key-pair** | ✅ Tested | Recommended for production |
| **Password** | ✅ Tested | For development only |
| **OAuth** | ⚠️ Known Issues | Not recommended |

### Environment-Specific Guidance

| Environment | Recommended Approach | Key Access |
|-------------|---------------------|------------|
| **Local IDE** | Direct DuckDB→Snowflake | Key file on disk |
| **SPCS / Remote Dev** | Direct DuckDB→Snowflake | Mount secret via service spec |
| **Workspace Notebooks** | **Python Bridge (7.3.1)** | No key needed - uses session |

### Why Python Bridge for Workspace Notebooks?

Workspace Notebooks (vNext) have limitations for private key access:
- `st.secrets` not supported (Streamlit not available)
- No service spec control (can't mount secrets)
- UDF workaround requires elevated privileges (CREATE SECRET, CREATE INTEGRATION)

The **Python Bridge** avoids these issues entirely by using the existing Snowpark session.

In [ ]:
%%R
# Install duckdb if not available (via micromamba - faster)
if (!requireNamespace("duckdb", quietly = TRUE)) {
    cat("Installing duckdb R package via micromamba...\n")
    system("/root/.local/share/mamba/bin/micromamba install -n r_env -c conda-forge r-duckdb -y",
           ignore.stdout = TRUE)
    .libPaths("/root/.local/share/mamba/envs/r_env/lib/R/library")
}

library(DBI)
library(duckdb)
library(dplyr)
library(dbplyr)

cat("Loading DuckDB with Snowflake extension...\n")

# Connect to DuckDB (in-memory for speed, or file for persistence)
duckdb_con <- dbConnect(duckdb::duckdb(), dbdir = ":memory:")

# Load the Snowflake extension
tryCatch({
    dbExecute(duckdb_con, "INSTALL snowflake FROM community")
    dbExecute(duckdb_con, "LOAD snowflake")
    cat("✓ Snowflake extension loaded\n")
}, error = function(e) {
    cat("✗ Error loading extension:", conditionMessage(e), "\n")
})

cat("DuckDB ready. Configure Snowflake secret in next cell.\n")

In [ ]:
# PY__duckdb_auth_prep
# Prepare DuckDB authentication for Snowflake extension
#
# Environment-specific behavior:
# - Local IDE: Uses key-pair from notebook_config.yaml
# - SPCS/Remote Dev: Mount secret via service spec, then read file
# - Workspace Notebooks: Skip direct connection, use Python Bridge (7.3.1)

import rpy2.robjects as ro

if ENV_TYPE == 'workspace':
    print("=" * 60)
    print("WORKSPACE NOTEBOOK - Authentication Setup")
    print("=" * 60)
    print("\nDirect DuckDB→Snowflake requires private key access.")
    print("Workspace Notebooks have limited options for secure key access:")
    print("  - st.secrets: Not supported (Streamlit not available)")
    print("  - Service spec: Not controllable from notebooks")
    print("  - UDF+EAI: Requires elevated privileges")
    print("\n→ RECOMMENDED: Use Python Bridge (Section 7.3.1)")
    print("  This uses the existing Snowpark session - no key needed.")
    ro.globalenv['duckdb_auth'] = ro.ListVector({'method': 'none'})
    
elif ENV_TYPE == 'spcs' or ENV_TYPE == 'remote_dev':
    # SPCS/Remote Dev: Check for mounted secret
    secret_path = '/secrets/snowflake_private_key'
    if os.path.exists(secret_path):
        with open(secret_path, 'r') as f:
            private_key = f.read()
        ro.globalenv['duckdb_auth'] = ro.ListVector({
            'method': 'keypair',
            'account': ENV_CONFIG.get('account', ''),
            'user': ENV_CONFIG.get('user', ''),
            'database': ENV_CONFIG.get('database', ''),
            'warehouse': ENV_CONFIG.get('warehouse', ''),
            'private_key': private_key
        })
        print("✓ Key-pair auth configured from mounted secret")
    else:
        print(f"⚠ No secret mounted at {secret_path}")
        print("  Add to service spec: snowflakeSecret with directoryPath: /secrets")
        ro.globalenv['duckdb_auth'] = ro.ListVector({'method': 'none'})
        
else:
    # Local IDE: Use key-pair from config file
    key_path = os.path.expanduser(ENV_CONFIG.get('private_key_path', ''))
    if os.path.exists(key_path):
        with open(key_path, 'r') as f:
            private_key = f.read()
        
        ro.globalenv['duckdb_auth'] = ro.ListVector({
            'method': 'keypair',
            'account': ENV_CONFIG.get('account', ''),
            'user': ENV_CONFIG.get('user', ''),
            'database': ENV_CONFIG.get('database', ''),
            'warehouse': ENV_CONFIG.get('warehouse', ''),
            'private_key': private_key
        })
        print("✓ Key-pair auth configured for DuckDB Snowflake extension")
        print(f"  Account: {ENV_CONFIG.get('account', '')}")
        print(f"  User: {ENV_CONFIG.get('user', '')}")
    else:
        print("⚠ Private key not found at:", key_path)
        print("  Configure 'private_key_path' in notebook_config.yaml")
        ro.globalenv['duckdb_auth'] = ro.ListVector({'method': 'none'})

In [ ]:
%%R
# Create DuckDB Snowflake secret (Local IDE only)

if (duckdb_auth$method == "none") {
    cat("DuckDB direct Snowflake connection not configured.\n")
    cat("\nFor Workspace Notebooks: Use Python Bridge (Section 7.3.1)\n")
    cat("For Local IDE: Configure key-pair auth in previous cell\n")
} else if (duckdb_auth$method == "keypair") {
    cat("Creating Snowflake secret with key-pair auth...\n")
    cat("  Account:", duckdb_auth$account, "\n")
    cat("  User:", duckdb_auth$user, "\n")
    
    secret_sql <- sprintf("
CREATE OR REPLACE SECRET snowflake_secret (
    TYPE snowflake,
    ACCOUNT '%s',
    USER '%s',
    DATABASE '%s',
    WAREHOUSE '%s',
    AUTH_TYPE 'key_pair',
    PRIVATE_KEY '%s'
)",
        duckdb_auth$account,
        duckdb_auth$user,
        duckdb_auth$database,
        duckdb_auth$warehouse,
        gsub("'", "''", duckdb_auth$private_key)
    )
    
    tryCatch({
        dbExecute(duckdb_con, secret_sql)
        cat("✓ Key-pair secret created successfully\n")
        cat("\nRun next cell to attach Snowflake database\n")
    }, error = function(e) {
        cat("✗ Error:", conditionMessage(e), "\n")
    })
}

In [ ]:
%%R
# Attach Snowflake as a catalog in DuckDB (Local IDE only)
# For Workspace Notebooks, skip to Section 7.3.1 Python Bridge

if (!exists("duckdb_auth") || is.null(duckdb_auth) || duckdb_auth$method == "none") {
    cat("Skipping - use Python Bridge (Section 7.3.1) for Workspace Notebooks\n")
} else {
    cat("Attaching Snowflake database...\n")
    
    tryCatch({
        dbExecute(duckdb_con, "ATTACH '' AS sf (TYPE snowflake, SECRET snowflake_secret, READ_ONLY)")
        cat("✓ Snowflake attached as 'sf' catalog\n\n")
        
        # List schemas
        cat("Available schemas:\n")
        schemas <- dbGetQuery(duckdb_con, 
            "SELECT schema_name FROM sf.information_schema.schemata ORDER BY schema_name LIMIT 10")
        rprint(schemas)
        
    }, error = function(e) {
        cat("✗ Error:", conditionMessage(e), "\n")
        cat("\nTroubleshooting:\n")
        cat("  - Verify account name format (e.g., 'xy12345' not full URL)\n")
        cat("  - Check private key is valid PKCS8 format\n")
        cat("  - Ensure public key is registered in Snowflake\n")
    })
}

### 5.3.1 Python Bridge (Recommended for Workspace Notebooks)

This approach queries Snowflake via Python Snowpark and transfers data to R/DuckDB for local analysis.

**Why use this?**
- Works reliably in both Workspace Notebooks and Local IDEs
- Uses the existing Snowpark session authentication
- No additional credential setup needed
- Ideal for dplyr/dbplyr workflows on fetched data

In [ ]:
# Python Bridge: Query Snowflake, analyze with R/DuckDB
# This example uses SNOWFLAKE_SAMPLE_DATA (available to all Snowflake accounts)

if session is None:
    print("No session available. Run Section 3.1 first!")
else:
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.conversion import localconverter
    
    # Query TPCH sample data
    # Note: Uses SNOWFLAKE_SAMPLE_DATA which is available to all accounts
    query = """
        SELECT O_ORDERKEY, O_CUSTKEY, O_ORDERSTATUS, 
               O_TOTALPRICE::FLOAT as O_TOTALPRICE, 
               O_ORDERDATE
        FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
        WHERE O_ORDERDATE >= '1995-01-01'
        LIMIT 10000
    """
    
    print("Querying SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS...")
    orders_df = session.sql(query).to_pandas()
    
    # Transfer to R environment
    with localconverter(ro.default_converter + pandas2ri.converter):
        r_orders = ro.conversion.py2rpy(orders_df)
        ro.globalenv['sf_orders'] = r_orders
    
    print(f"✓ Transferred {len(orders_df):,} rows to R as 'sf_orders'")
    print(f"  Columns: {', '.join(orders_df.columns)}")
    print("\nUse %%R cells to analyze with dplyr:")
    print("  library(dplyr)")
    print("  sf_orders %>% group_by(O_ORDERSTATUS) %>% summarize(n = n())")

In [ ]:
%%R
# Analyze data transferred via Python bridge
# This cell works in Workspace Notebooks without DuckDB

if (exists("sf_orders")) {
    library(dplyr)
    
    # Convert date column if needed (pandas dates may come through differently)
    sf_orders <- sf_orders %>%
        mutate(
            O_ORDERDATE = as.Date(O_ORDERDATE),
            order_year = as.character(format(O_ORDERDATE, "%Y"))
        )
    
    result <- sf_orders %>%
        group_by(order_year, O_ORDERSTATUS) %>%
        summarise(
            orders = n(),
            total_value = sum(O_TOTALPRICE, na.rm = TRUE),
            .groups = "drop"
        ) %>%
        arrange(order_year, desc(orders))
    
    cat("Order analysis using dplyr (via Python bridge):\n")
    rprint(result)
} else {
    cat("Note: sf_orders not found. Run the Python bridge cell above first.\n")
    cat("Or use the DuckDB approach if in local IDE.\n")
}

## 5.4 Query Snowflake with dplyr

The recommended pattern for dplyr workflows:
1. **Direct SQL** for fetching data from Snowflake
2. **Cache locally** in DuckDB for iterative analysis
3. **Use dplyr** on local cached tables

**Important**: Use 2-part table names (`sf.schema.table`) when database is set in the secret.

In [ ]:
%%R
# Pattern 1: Direct SQL Query to Snowflake
# Best for: simple aggregations, data exploration
# NOTE: Requires DuckDB ATTACH (Local IDE only)

# Check if sf catalog exists (only available in Local IDE with key-pair auth)
sf_exists <- tryCatch({
    dbGetQuery(duckdb_con, "SELECT 1 FROM duckdb_databases() WHERE database_name = 'sf'")
    TRUE
}, error = function(e) FALSE)

if (!sf_exists || nrow(dbGetQuery(duckdb_con, "SELECT 1 FROM duckdb_databases() WHERE database_name = 'sf'")) == 0) {
    cat("Note: 'sf' catalog not attached (Workspace Notebook mode)\n")
    cat("Use the Python Bridge approach in Section 7.3.1 instead.\n")
} else {
    cat("Direct SQL query to Snowflake...\n\n")
    
    tryCatch({
        customers <- dbGetQuery(duckdb_con, "
            SELECT C_MKTSEGMENT, COUNT(*) as customers, ROUND(AVG(C_ACCTBAL), 2) as avg_balance
            FROM sf.tpch_sf1.customer
            GROUP BY C_MKTSEGMENT
            ORDER BY customers DESC
        ")
        
        cat("Customer analysis by market segment:\n")
        rprint(customers)
        
    }, error = function(e) {
        cat("Error:", conditionMessage(e), "\n")
    })
}

In [ ]:
%%R
# Pattern 2: Cache locally, then use dplyr
# Best for: complex analysis, multiple operations on same data
# NOTE: Requires DuckDB ATTACH (Local IDE only)

sf_exists <- tryCatch({
    nrow(dbGetQuery(duckdb_con, "SELECT 1 FROM duckdb_databases() WHERE database_name = 'sf'")) > 0
}, error = function(e) FALSE)

if (!sf_exists) {
    cat("Note: 'sf' catalog not attached (Workspace Notebook mode)\n")
    cat("Use the Python Bridge in Section 7.3.1 - data is cached in sf_orders.\n")
} else {
    cat("Caching Snowflake data locally...\n\n")
    
    tryCatch({
        # Pull data into local DuckDB table
        dbExecute(duckdb_con, "
            CREATE OR REPLACE TABLE local_orders AS
            SELECT O_ORDERKEY, O_CUSTKEY, O_ORDERSTATUS, 
                   O_TOTALPRICE, O_ORDERDATE, O_ORDERPRIORITY
            FROM sf.tpch_sf1.orders
            LIMIT 50000
        ")
        
        # Use dplyr on local table
        local_orders <- tbl(duckdb_con, "local_orders")
        
        result <- local_orders %>%
            group_by(O_ORDERSTATUS, O_ORDERPRIORITY) %>%
            summarise(
                count = n(),
                avg_price = mean(O_TOTALPRICE, na.rm = TRUE),
                .groups = "drop"
            ) %>%
            collect()
        
        cat("Order analysis (cached locally):\n")
        rprint(result)
        
    }, error = function(e) {
        cat("Error:", conditionMessage(e), "\n")
    })
}

## 5.5 Advanced Patterns

Additional patterns for DuckDB + Snowflake workflows.

In [ ]:
%%R
# Pattern 3: Join local cached tables
# Best for: multi-table analysis after caching
# NOTE: Requires DuckDB ATTACH (Local IDE only)

sf_exists <- tryCatch({
    nrow(dbGetQuery(duckdb_con, "SELECT 1 FROM duckdb_databases() WHERE database_name = 'sf'")) > 0
}, error = function(e) FALSE)

if (!sf_exists) {
    cat("Note: 'sf' catalog not attached (Workspace Notebook mode)\n")
    cat("For multi-table joins, use Python Bridge to fetch each table.\n")
} else {
    cat("Caching and joining tables...\n\n")
    
    tryCatch({
        # Cache customers
        dbExecute(duckdb_con, "
            CREATE OR REPLACE TABLE local_customers AS
            SELECT C_CUSTKEY, C_NAME, C_MKTSEGMENT, C_NATIONKEY
            FROM sf.tpch_sf1.customer
        ")
        
        # Join with previous local_orders
        result <- dbGetQuery(duckdb_con, "
            SELECT c.C_MKTSEGMENT, 
                   COUNT(DISTINCT o.O_ORDERKEY) as orders,
                   ROUND(SUM(o.O_TOTALPRICE), 2) as total_sales
            FROM local_orders o
            JOIN local_customers c ON o.O_CUSTKEY = c.C_CUSTKEY
            GROUP BY c.C_MKTSEGMENT
            ORDER BY total_sales DESC
        ")
        
        cat("Sales by market segment (joined tables):\n")
        rprint(result)
        
    }, error = function(e) {
        cat("Error:", conditionMessage(e), "\n")
    })
}

In [ ]:
%%R
# Pattern 4: Window functions with dplyr
# Best for: rankings, running totals, lag/lead analysis
# NOTE: Requires local cached data from previous cells

if (!dbExistsTable(duckdb_con, "local_orders")) {
    cat("Note: local_orders table not found\n")
    cat("Run Pattern 2 first to cache data, or use Python Bridge.\n")
} else {
    cat("Window function analysis...\n\n")
    
    tryCatch({
        local_orders <- tbl(duckdb_con, "local_orders")
        
        result <- local_orders %>%
            group_by(O_ORDERPRIORITY) %>%
            mutate(
                priority_rank = row_number(),
                running_total = cumsum(O_TOTALPRICE)
            ) %>%
            filter(priority_rank <= 3) %>%
            select(O_ORDERPRIORITY, O_ORDERKEY, O_TOTALPRICE, 
                   priority_rank, running_total) %>%
            arrange(O_ORDERPRIORITY, priority_rank) %>%
            collect()
        
        cat("Top 3 orders per priority (window functions):\n")
        rprint(head(result, 15))
        
    }, error = function(e) {
        cat("Error:", conditionMessage(e), "\n")
    })
}

## 5.6 Cleanup

Close the DuckDB connection when done.

In [ ]:
%%R
# Cleanup: Disconnect from DuckDB
# Uncomment to close connection

# dbDisconnect(duckdb_con)
# cat("DuckDB connection closed\n")

---

# Section 6: Iceberg Integration via Horizon Catalog

This section demonstrates accessing **Snowflake-managed Iceberg tables** from R/DuckDB using the Horizon Catalog REST API.

### Architecture

```
R (DuckDB)  →  Horizon Catalog REST API  →  Snowflake-managed Iceberg Table
                     ↓
            Vended S3 credentials → Direct data file access
```

### Key Features (GA)

- **External engine reads via Horizon are GA** (Build '26)
- **Snowflake-managed internal storage** - No external cloud storage required!
- **Warehouse-free reads** - Cloud Services billing only
- **Vended credentials** - Horizon provides temporary S3-style credentials

### Approach

1. **Create External Volume** with `STORAGE_PROVIDER = 'SNOWFLAKE'` (internal storage)
2. **Create Iceberg table** using CTAS from sample data
3. **Generate PAT/JWT** for Horizon API authentication
4. **Query from DuckDB** via Iceberg REST catalog

## 6.1 Create Iceberg Table on Internal Storage (One-Time Setup)

Create a Snowflake-managed Iceberg table using **internal Snowflake storage** (no external S3/Azure/GCS required).

**Steps:**
1. Create an External Volume with `STORAGE_PROVIDER = 'SNOWFLAKE'`
2. Create an Iceberg table using CTAS from TPCH sample data

**Required Privileges:**
- `CREATE EXTERNAL VOLUME` on account (or use existing volume)
- `CREATE ICEBERG TABLE` on schema

**Note:** Internal storage for Iceberg is GA in most regions. If you encounter errors, you may need feature enablement.

In [ ]:
# PY__create_iceberg_table
# Create Snowflake-managed Iceberg table on internal storage
# Uses ICEBERG_CONFIG from notebook_config.yaml

if session is None:
    print("No session available. Run Section 3.1 first!")
else:
    # Configuration from YAML
    iceberg_table = ICEBERG_CONFIG.get('test_table_name', 'NATION_ICEBERG')
    external_vol = ICEBERG_CONFIG.get('external_volume', 'iceberg_internal_vol')
    target_db = DEFAULTS.get('database', 'SIMON')  # Use user's DB, not SNOWFLAKE_SAMPLE_DATA
    target_schema = DEFAULTS.get('schema', 'PUBLIC')
    
    # Use user's own database for Iceberg table (need write access)
    if target_db == 'SNOWFLAKE_SAMPLE_DATA':
        target_db = CONN_CONFIG.get('user', 'SIMON')  # Fall back to username as DB
        target_schema = 'PUBLIC'
    
    print("=" * 60)
    print("STEP 1: Create External Volume with Internal Storage")
    print("=" * 60)
    
    # Create external volume with Snowflake-managed internal storage
    create_vol_sql = f"""
    CREATE EXTERNAL VOLUME IF NOT EXISTS {external_vol}
        STORAGE_LOCATIONS = (
            (NAME = 'sf_internal', STORAGE_PROVIDER = 'SNOWFLAKE')
        )
        COMMENT = 'Internal storage for Iceberg demo tables'
    """
    
    try:
        session.sql(create_vol_sql).collect()
        print(f"✓ External volume '{external_vol}' ready")
    except Exception as e:
        if 'already exists' in str(e).lower():
            print(f"✓ External volume '{external_vol}' already exists")
        else:
            print(f"⚠ Volume creation failed: {e}")
            print("  You may need CREATE EXTERNAL VOLUME privilege or feature enablement")
    
    print("\n" + "=" * 60)
    print("STEP 2: Create Iceberg Table from TPCH Sample Data")
    print("=" * 60)
    
    full_table_name = f"{target_db}.{target_schema}.{iceberg_table}"
    print(f"Creating: {full_table_name}")
    print(f"External volume: {external_vol}")
    
    create_table_sql = f"""
    CREATE OR REPLACE ICEBERG TABLE {full_table_name}
        EXTERNAL_VOLUME = '{external_vol}'
        CATALOG = 'SNOWFLAKE'
        BASE_LOCATION = '{iceberg_table.lower()}/'
        AS SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION
    """
    
    try:
        session.sql(create_table_sql).collect()
        print(f"✓ Iceberg table '{iceberg_table}' created successfully!")
        
        # Verify
        count = session.sql(f"SELECT COUNT(*) as cnt FROM {full_table_name}").collect()[0]['CNT']
        print(f"✓ Table contains {count} rows")
        
        # Store for later cells
        ICEBERG_FULL_TABLE = full_table_name
        print(f"\n✓ Ready for Horizon Catalog queries in Section 8.2+")
        
    except Exception as e:
        print(f"✗ Table creation failed: {e}")
        print("\nPossible issues:")
        print("  - Need CREATE ICEBERG TABLE privilege")
        print("  - Internal storage may require feature enablement")
        print("  - Check if external volume supports internal storage")

## 6.2 Horizon Catalog Authentication

Get an access token for the Horizon Catalog REST API.

**Authentication Options:**
1. **PAT (Personal Access Token)** - Easiest, already set up in Section 3
2. **JWT (Key-pair)** - More secure, requires private key

For Workspace Notebooks, we'll use a PAT (if available) or fall back to JWT.

In [ ]:
# PY__horizon_auth
# Get access token for Horizon Catalog
# In Workspace: Try to use session token
# In Local IDE: Use JWT from private key

import os

horizon_access_token = None

if ENV_TYPE == 'workspace':
    print("=" * 60)
    print("WORKSPACE NOTEBOOK - Horizon Authentication")
    print("=" * 60)
    
    # Try to extract session token from Snowpark session
    try:
        # Method 1: Direct token access from REST client
        if hasattr(session, '_conn') and hasattr(session._conn, '_rest'):
            rest = session._conn._rest
            if hasattr(rest, '_token'):
                horizon_access_token = rest._token
                print("✓ Extracted session token from Snowpark session")
            elif hasattr(rest, 'token'):
                horizon_access_token = rest.token
                print("✓ Extracted session token (alt method)")
        
        # Method 2: From session file (SPCS)
        if not horizon_access_token:
            token_file = '/snowflake/session/token'
            if os.path.exists(token_file):
                with open(token_file, 'r') as f:
                    horizon_access_token = f.read().strip()
                print(f"✓ Read token from {token_file}")
        
        # Method 3: Environment variable
        if not horizon_access_token:
            horizon_access_token = os.environ.get('SNOWFLAKE_TOKEN') or os.environ.get('SF_TOKEN')
            if horizon_access_token:
                print("✓ Got token from environment variable")
                
    except Exception as e:
        print(f"⚠ Token extraction failed: {e}")
    
    if not horizon_access_token:
        print("\n⚠ Could not extract session token automatically.")
        print("\nOptions:")
        print("  1. Create a PAT (Section 3.4), then set:")
        print("     horizon_access_token = 'your_pat_token'")
        print("  2. Use Python Bridge (Section 7.3.1) - no token needed")
        
elif ENV_TYPE == 'local':
    # Local IDE: Generate JWT
    import jwt
    import time
    import hashlib
    import base64
    import requests
    from cryptography.hazmat.primitives import serialization
    
    key_path = os.path.expanduser(ENV_CONFIG.get('private_key_path', ''))
    
    if os.path.exists(key_path):
        print("Generating JWT for Horizon authentication...")
        
        with open(key_path, 'rb') as f:
            private_key = serialization.load_pem_private_key(f.read(), password=None)
        
        public_key = private_key.public_key()
        public_key_bytes = public_key.public_bytes(
            encoding=serialization.Encoding.DER,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
        fingerprint = hashlib.sha256(public_key_bytes).digest()
        fingerprint_b64 = base64.b64encode(fingerprint).decode()
        
        account = ENV_CONFIG.get('account', '').upper()
        user = ENV_CONFIG.get('user', '').upper()
        
        now = int(time.time())
        payload = {
            'iss': f'{account}.{user}.SHA256:{fingerprint_b64}',
            'sub': f'{account}.{user}',
            'iat': now,
            'exp': now + 3600
        }
        
        token = jwt.encode(payload, private_key, algorithm='RS256')
        
        # Exchange for access token
        token_url = f"https://{account}.snowflakecomputing.com/oauth/token"
        
        try:
            resp = requests.post(token_url, data={
                'grant_type': 'urn:ietf:params:oauth:grant-type:jwt-bearer',
                'assertion': token,
                'scope': 'session:role:SYSADMIN'
            })
            
            if resp.status_code == 200:
                horizon_access_token = resp.json().get('access_token')
                print(f"✓ Access token obtained")
            else:
                print(f"⚠ Token exchange failed: {resp.status_code}")
        except Exception as e:
            print(f"⚠ Error: {e}")
    else:
        print(f"⚠ Private key not found: {key_path}")

if horizon_access_token:
    print(f"\n✓ Token ready ({len(horizon_access_token)} chars)")
else:
    print("\n⚠ No token available")

## 6.3 Query Horizon Catalog Metadata

Use the Horizon REST API to list namespaces and find our Iceberg table.

**Endpoint**: `https://<account>.snowflakecomputing.com/polaris/api/catalog/v1/`

In [ ]:
# PY__query_horizon_api
# Query Horizon Catalog REST API
import requests
import json as json_module

def query_horizon(endpoint, access_token, account):
    """Query the Horizon Catalog REST API."""
    base_url = f"https://{account}.snowflakecomputing.com/polaris/api/catalog/v1"
    
    response = requests.get(
        f"{base_url}/{endpoint}",
        headers={
            'Authorization': f'Bearer {access_token}',
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    )
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text[:200]}")
        return None

# Check if we have a token
if 'horizon_access_token' not in dir() or horizon_access_token is None:
    print("⚠ No access token. Run Section 8.2 first or set horizon_access_token manually.")
else:
    account = ENV_CONFIG.get('account', '').upper()
    
    print("=" * 60)
    print("Horizon Catalog API Queries")
    print("=" * 60)
    
    # List namespaces (databases)
    print("\n1. Listing namespaces (databases)...")
    namespaces = query_horizon("namespaces", horizon_access_token, account)
    if namespaces:
        print(f"   Found {len(namespaces.get('namespaces', []))} namespaces")
        for ns in namespaces.get('namespaces', [])[:5]:
            print(f"   - {ns}")
        if len(namespaces.get('namespaces', [])) > 5:
            print(f"   ... and {len(namespaces.get('namespaces', [])) - 5} more")
    
    # List tables in our target database
    target_db = DEFAULTS.get('database', 'SIMON')
    if target_db == 'SNOWFLAKE_SAMPLE_DATA':
        target_db = CONN_CONFIG.get('user', 'SIMON')
    
    print(f"\n2. Listing tables in {target_db}...")
    tables = query_horizon(f"namespaces/{target_db}/tables", horizon_access_token, account)
    if tables:
        print(f"   Found {len(tables.get('identifiers', []))} tables")
        for t in tables.get('identifiers', []):
            print(f"   - {t.get('namespace', [''])[0]}.{t.get('name', '')}")
    
    # Get our Iceberg table metadata
    iceberg_table = ICEBERG_CONFIG.get('test_table_name', 'NATION_ICEBERG')
    target_schema = DEFAULTS.get('schema', 'PUBLIC')
    
    print(f"\n3. Getting metadata for {target_db}.{target_schema}.{iceberg_table}...")
    table_meta = query_horizon(
        f"namespaces/{target_db}.{target_schema}/tables/{iceberg_table}", 
        horizon_access_token, account
    )
    if table_meta:
        print(f"   ✓ Table found!")
        print(f"   Format: {table_meta.get('metadata', {}).get('format-version', 'unknown')}")
        print(f"   Location: {table_meta.get('metadata-location', 'unknown')[:50]}...")

## 6.4 Query Iceberg Table from DuckDB

Use DuckDB's Iceberg extension to connect to Horizon Catalog and query the table we created.

**Architecture**:
```
DuckDB (R)  →  Horizon REST API  →  Vended S3 credentials  →  Data files
```

**Requirements**:
- Access token from Section 8.2
- DuckDB iceberg extension (installed below)

In [ ]:
# PY__iceberg_duckdb_setup
# Prepare config for R DuckDB Iceberg cell
import rpy2.robjects as ro

# Get config values
account = ENV_CONFIG.get('account', '').upper()
target_db = DEFAULTS.get('database', 'SIMON')
if target_db == 'SNOWFLAKE_SAMPLE_DATA':
    target_db = CONN_CONFIG.get('user', 'SIMON')
target_schema = DEFAULTS.get('schema', 'PUBLIC')
if target_schema == 'TPCH_SF1':
    target_schema = 'PUBLIC'
iceberg_table = ICEBERG_CONFIG.get('test_table_name', 'NATION_ICEBERG')

# Get access token
token = horizon_access_token if 'horizon_access_token' in dir() and horizon_access_token else ''

# Create R ListVector with config
ro.globalenv['iceberg_r_config'] = ro.ListVector({
    'access_token': token,
    'account': account,
    'database': target_db,
    'schema': target_schema,
    'table_name': iceberg_table
})

print(f"✓ Config prepared for R:")
print(f"  Account: {account}")
print(f"  Database: {target_db}")
print(f"  Schema: {target_schema}")
print(f"  Table: {iceberg_table}")
print(f"  Token: {'set' if token else 'NOT SET - run Section 8.2'}")

In [ ]:
%%R
# DuckDB Iceberg Integration
# Query Snowflake-managed Iceberg table via Horizon Catalog

library(DBI)
library(duckdb)

# Connect to DuckDB
cat("Setting up DuckDB with Iceberg extension...\n\n")
iceberg_con <- dbConnect(duckdb::duckdb(), dbdir = ":memory:")

# Try to install and load extensions
extensions_ok <- TRUE

tryCatch({
    # Install required extensions
    dbExecute(iceberg_con, "INSTALL httpfs")
    dbExecute(iceberg_con, "INSTALL avro")  # Required by iceberg
    dbExecute(iceberg_con, "INSTALL iceberg")
    
    # Load them
    dbExecute(iceberg_con, "LOAD httpfs")
    dbExecute(iceberg_con, "LOAD iceberg")
    cat("✓ Extensions loaded\n")
}, error = function(e) {
    cat("⚠ Extension installation failed:\n")
    cat("  ", conditionMessage(e), "\n")
    cat("\nThe DuckDB iceberg extension requires 'avro' which may not be\n")
    cat("available in this environment.\n")
    cat("\n→ ALTERNATIVE: Use Python DuckDB (cell below) or Python Bridge (Section 7.3.1)\n")
    extensions_ok <<- FALSE
})

if (extensions_ok && exists("iceberg_r_config")) {
    # Extract config
    access_token <- iceberg_r_config$access_token
    account <- iceberg_r_config$account
    target_db <- iceberg_r_config$database
    target_schema <- iceberg_r_config$schema
    iceberg_table <- iceberg_r_config$table_name
    
    cat("\nConfiguration:\n")
    cat("  Account:", account, "\n")
    cat("  Database:", target_db, "\n")
    cat("  Schema:", target_schema, "\n")
    cat("  Table:", iceberg_table, "\n")
    
    if (!is.null(access_token) && access_token != "") {
        # Create Iceberg secret
        cat("\nCreating Iceberg secret...\n")
        secret_sql <- sprintf("
            CREATE OR REPLACE SECRET horizon_secret (
                TYPE iceberg,
                TOKEN '%s'
            )
        ", access_token)
        
        tryCatch({
            dbExecute(iceberg_con, secret_sql)
            cat("✓ Secret created\n")
            
            # Attach Horizon Catalog
            cat("\nAttaching Horizon Catalog...\n")
            endpoint <- sprintf("https://%s.snowflakecomputing.com/polaris/api/catalog", account)
            
            attach_sql <- sprintf("
                ATTACH '%s' AS horizon (
                    TYPE iceberg,
                    ENDPOINT '%s',
                    SECRET horizon_secret
                )
            ", target_db, endpoint)
            
            dbExecute(iceberg_con, attach_sql)
            cat("✓ Horizon Catalog attached\n")
            
            # Query the table
            cat("\nQuerying", iceberg_table, "...\n")
            query_sql <- sprintf(
                "SELECT * FROM horizon.\"%s\".%s LIMIT 10",
                target_schema, iceberg_table
            )
            
            result <- dbGetQuery(iceberg_con, query_sql)
            cat("\n✓ Query successful! Returned", nrow(result), "rows:\n\n")
            print(result)
            
        }, error = function(e) {
            cat("\n⚠ Error:", conditionMessage(e), "\n")
        })
    } else {
        cat("\n⚠ No access token. Run Section 8.2 first.\n")
    }
} else if (!exists("iceberg_r_config")) {
    cat("\n⚠ Config not found. Run PY__iceberg_duckdb_setup first.\n")
}

cat("\nDone.\n")

In [ ]:
# PY__iceberg_duckdb_python
# Alternative: Use Python DuckDB for Iceberg

# Install duckdb if not available
try:
    import duckdb
except ImportError:
    print("Installing duckdb...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'duckdb', '-q'])
    import duckdb
    print("✓ duckdb installed")

if 'horizon_access_token' not in dir() or not horizon_access_token:
    print("⚠ No access token. Run Section 8.2 first.")
else:
    # Get config
    account = ENV_CONFIG.get('account', '').upper()
    target_db = DEFAULTS.get('database', CONN_CONFIG.get('user', 'SIMON'))
    if target_db == 'SNOWFLAKE_SAMPLE_DATA':
        target_db = CONN_CONFIG.get('user', 'SIMON')
    target_schema = 'PUBLIC' if DEFAULTS.get('schema') == 'TPCH_SF1' else DEFAULTS.get('schema', 'PUBLIC')
    iceberg_table = ICEBERG_CONFIG.get('test_table_name', 'NATION_ICEBERG')
    
    print("Setting up Python DuckDB with Iceberg extension...")
    print(f"  Account: {account}")
    print(f"  Database: {target_db}")
    print(f"  Schema: {target_schema}")
    print(f"  Table: {iceberg_table}")
    
    try:
        con = duckdb.connect(':memory:')
        
        # Install extensions
        print("\nInstalling extensions...")
        con.execute("INSTALL httpfs")
        con.execute("INSTALL iceberg")
        con.execute("LOAD httpfs")
        con.execute("LOAD iceberg")
        print("✓ Extensions loaded")
        
        # Create secret
        con.execute(f"""
            CREATE OR REPLACE SECRET horizon_secret (
                TYPE iceberg,
                TOKEN '{horizon_access_token}'
            )
        """)
        print("✓ Secret created")
        
        # Attach Horizon
        endpoint = f"https://{account}.snowflakecomputing.com/polaris/api/catalog"
        print(f"\nAttaching Horizon Catalog...")
        print(f"  Endpoint: {endpoint}")
        
        con.execute(f"""
            ATTACH '{target_db}' AS horizon (
                TYPE iceberg,
                ENDPOINT '{endpoint}',
                SECRET horizon_secret
            )
        """)
        print("✓ Horizon Catalog attached")
        
        # Query
        print(f"\nQuerying {target_schema}.{iceberg_table}...")
        result = con.execute(f'SELECT * FROM horizon."{target_schema}".{iceberg_table} LIMIT 10').fetchdf()
        print(f"\n✓ Query successful! {len(result)} rows:")
        display(result)
        
        con.close()
        
    except Exception as e:
        print(f"\n⚠ Error: {e}")
        print("\nThis may be due to:")
        print("  - Extension not available in this environment")
        print("  - Vended credentials not configured")
        print("  - Authentication issue with Horizon")
        print("\n→ Use Python Bridge (Section 7.3.1) as reliable fallback")

## 6.5 Recommended Alternative: Snowflake + DuckDB Hybrid

Until full Iceberg REST catalog support is available, use the working DuckDB Snowflake extension approach from Section 7:

1. **Query Snowflake via ADBC** (using DuckDB Snowflake extension)
2. **Cache results locally** in DuckDB
3. **Use dplyr/dbplyr** on the local cache

This provides the same benefits (local processing, R ecosystem) with full support today.

In [ ]:
%%R
# Hybrid Approach: Best of Both Worlds
# Use the working DuckDB + Snowflake pattern for Iceberg-like benefits

# Prerequisites: Run Section 7.3 first to create duckdb_con

# Example: Query Snowflake Iceberg table, cache locally
# (Even though it's an Iceberg table in Snowflake, query via SQL works!)

# Check if duckdb_con exists
if (!exists("duckdb_con")) {
    cat("duckdb_con not found.\n")
    cat("Run Section 7.3 first to set up DuckDB connection.\n")
} else {
    # Query the Iceberg table via standard Snowflake SQL
    dbExecute(duckdb_con, "
        CREATE OR REPLACE TABLE nation_iceberg_local AS 
        SELECT * FROM sf.PUBLIC.NATION_ICEBERG
    ")
    
    # Now use dplyr on the local cache
    library(dplyr)
    library(dbplyr)
    
    tbl(duckdb_con, 'nation_iceberg_local') %>%
        group_by(N_REGIONKEY) %>%
        summarise(
            nations = n(),
            sample_name = first(N_NAME)
        ) %>%
        collect() %>%
        print()
    
    cat("\n✓ Hybrid pattern complete.\n")
}

---

## Troubleshooting

### Common Issues

| Issue | Solution |
|-------|----------|
| `ModuleNotFoundError: No module named 'rpy2'` | Run Section 1.2 to install rpy2 |
| `R.version.string` returns error | Verify PATH and R_HOME are set correctly |
| ADBC `auth_pat` error | Ensure PAT was created and stored in `SNOWFLAKE_PAT` |
| Network policy error | PAT may need `MINS_TO_BYPASS_NETWORK_POLICY_REQUIREMENT` |
| `adbcsnowflake` not found | Ensure setup script ran with `--adbc` flag |
| Setup script fails | Check `setup_r.log` for detailed error messages |
| `r_sf_con` not found | Run `get_snowflake_connection()` to create connection |

### Run Full Diagnostics

In [ ]:
# Comprehensive diagnostic check
from r_helpers import print_diagnostics
print_diagnostics()

In [ ]:
# Environment diagnostics
import os
import shutil

print("Quick Environment Check:")
print(f"  R_HOME: {os.environ.get('R_HOME', 'NOT SET')}")
print(f"  R binary: {shutil.which('R') or 'NOT FOUND'}")
print(f"  SNOWFLAKE_ACCOUNT: {os.environ.get('SNOWFLAKE_ACCOUNT', 'NOT SET')}")
print(f"  SNOWFLAKE_PAT: {'SET' if os.environ.get('SNOWFLAKE_PAT') else 'NOT SET'}")

In [ ]:
# View setup log if something went wrong
# !tail -50 setup_r.log